<a href="https://colab.research.google.com/github/alunfes/GoogleColabProjects/blob/main/MnistDigitsCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms

In [11]:
# MNISTデータセットを読み込み、トレーニング用とテスト用に分割
mnist_data = datasets.MNIST(root="./data", train=True, download=True, transform=transforms.ToTensor())
train_data, test_data = torch.utils.data.random_split(mnist_data, [48000, 12000])  # トレーニングデータ48000枚、テストデータ12000枚

# DataLoaderを使用してデータをバッチ処理する
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

# Get the size of the first image in the training data
first_image, _ = train_data[0]
image_size = first_image.shape
print("Image size:", image_size)

Image size: torch.Size([1, 28, 28])


In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.25)
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Apply first convolutional layer, followed by activation and pooling
        x = self.pool(torch.relu(self.conv1(x)))
        # Apply second convolutional layer, followed by activation and pooling
        x = self.pool(torch.relu(self.conv2(x)))
        # Flatten the output from convolutional layers
        x = torch.flatten(x, 1)
        # Apply dropout
        x = self.dropout(x)
        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
# Initialize the model
model = CNN()
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Training loop
epochs = 50  # Increased the number of epochs
batch_size = 64  # Use mini-batch training

for epoch in range(epochs):
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")
# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
print(f"Accuracy on test set: {(correct / total) * 100:.2f}%")


Epoch 1/50, Loss: 0.2724631878150006
Epoch 2/50, Loss: 0.08430924515984953
Epoch 3/50, Loss: 0.06312953585168968
Epoch 4/50, Loss: 0.049992486560717225
Epoch 5/50, Loss: 0.04584702729992569
Epoch 6/50, Loss: 0.03827630283179072
Epoch 7/50, Loss: 0.034877867130097
Epoch 8/50, Loss: 0.031063458256151838


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_directory = '/content/drive/My Drive/Model/'
torch.save(model.state_dict(), model_directory+'digits_cnn.pth')